# Creating and Valuing a CDS Contract

Creates a CDS contract and does a valuation and calculates risk measures

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from financepy.finutils import *
from financepy.products.libor import *
from financepy.products.credit import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 19:38 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



## Creating a CDS Contract

In [2]:
tradeDate = FinDate(2019, 8, 9)
effectiveDate = tradeDate
maturityDate = FinDate(2029, 6, 20)
cdsCoupon = 0.0150
notional = ONE_MILLION
longProtection = False
tradeDate = FinDate(2019, 8, 9)

In [3]:
cdsContract = FinCDS(effectiveDate, maturityDate, cdsCoupon, notional, longProtection)

## Build Libor Curve

In [4]:
settlementDate = effectiveDate

In [6]:
dcType = FinDayCountTypes.ACT_360
depo1 = FinLiborDeposit(settlementDate, "1M", 0.022009, dcType)
depo2 = FinLiborDeposit(settlementDate, "2M", 0.022138, dcType)
depo3 = FinLiborDeposit(settlementDate, "3M", 0.021810, dcType)
depo4 = FinLiborDeposit(settlementDate, "6M", 0.020503, dcType)
depo5 = FinLiborDeposit(settlementDate, "12M", 0.019930, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swapType = FinLiborSwapTypes.PAYER

swap1 = FinLiborSwap(settlementDate,"2Y",swapType,0.015910,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"3Y",swapType,0.014990,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"4Y",swapType,0.014725,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"5Y",swapType,0.014640,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"6Y",swapType,0.014800,fixedFreq,dcType)
swap6 = FinLiborSwap(settlementDate,"7Y",swapType,0.014995,fixedFreq,dcType)
swap7 = FinLiborSwap(settlementDate,"8Y",swapType,0.015180,fixedFreq,dcType)
swap8 = FinLiborSwap(settlementDate,"9Y",swapType,0.015610,fixedFreq,dcType)
swap9 = FinLiborSwap(settlementDate,"10Y",swapType,0.0159880,fixedFreq,dcType)
swap10 = FinLiborSwap(settlementDate,"12Y",swapType,0.016430,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

# Build a CDS Curve

In [7]:
cds1 = FinCDS(settlementDate, "1Y", 0.0200)
cds2 = FinCDS(settlementDate, "2Y", 0.0220)
cds3 = FinCDS(settlementDate, "3Y", 0.0250)
cds4 = FinCDS(settlementDate, "4Y", 0.0275)
cds5 = FinCDS(settlementDate, "5Y", 0.0290)
cds6 = FinCDS(settlementDate, "7Y", 0.0300)
cds7 = FinCDS(settlementDate, "10Y", 0.0310)
cds8 = FinCDS(settlementDate, "15Y", 0.0315)

In [8]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [9]:
recoveryRate = 0.40

In [10]:
issuerCurve = FinCDSCurve(settlementDate, cdss, liborCurve, recoveryRate)

In [11]:
print(issuerCurve)

OBJECT TYPE: FinCDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 1.1178082,  0.9629280
 2.1178082,  0.9241524
 3.1178082,  0.8756501
 4.1178082,  0.8234894
 5.1205479,  0.7743682
 7.1205479,  0.6920884
10.1232877,  0.5813178
15.1260274,  0.4389424


# Valuation

In [13]:
spd = cdsContract.parSpread(settlementDate, issuerCurve, recoveryRate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  309.40693 bp


In [14]:
v = cdsContract.value(settlementDate, issuerCurve, recoveryRate)

In [15]:
full_pv = v['full_pv'] 
clean_pv = v['clean_pv']

In [16]:
print("FULL VALUE  %12.2f"% full_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

FULL VALUE    -115810.02
CLEAN VALUE   -117893.35


In [17]:
cleanp = cdsContract.cleanPrice(settlementDate, issuerCurve, recoveryRate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE    88.210959


In [18]:
accruedDays = cdsContract.accruedDays()
print("ACCRUED_DAYS", accruedDays)

ACCRUED_DAYS 50


In [19]:
accruedInterest = cdsContract.accruedInterest()
print("ACCRUED_COUPON", accruedInterest)

ACCRUED_COUPON 2083.3333333333335


In [20]:
protPV = cdsContract.protectionLegPV(settlementDate, issuerCurve, recoveryRate)
print("PROTECTION_PV", protPV)

PROTECTION_PV 228829.57245128392


In [21]:
premPV = cdsContract.premiumLegPV(settlementDate, issuerCurve, recoveryRate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 113019.55526164455


In [22]:
rpv01 = cdsContract.riskyPV01(settlementDate, issuerCurve)
print("FULL_RPV01", rpv01['full_rpv01'])
print("CLEAN_RPV01", rpv01['clean_rpv01'])

FULL_RPV01 7.534637017442971
CLEAN_RPV01 7.395748128554081


In [23]:
cdsContract.riskyPV01(settlementDate, issuerCurve)

{'full_rpv01': 7.534637017442971, 'clean_rpv01': 7.395748128554081}

## Risk Measures

In [24]:
creditDV01 = cdsContract.creditDV01(settlementDate, issuerCurve, recoveryRate)

In [25]:
creditDV01

-650.8402049520955

Copyright (c) 2020 Dominic O'Kane